# Figure 8. Compare vaccine strains to estimated and observed closest strains to the future

Observed distance to natural H3N2 populations one year into the future for each vaccine strain (green) and the observed (blue) and estimated (orange) closest strains to the future at the corresponding timepoints. Vaccine strains were assigned to the validation or test timepoint closest to the date they were selected by the WHO. The distance to the future of each vaccine strain was calculated from their amino acid sequences and the frequencies and sequences of the corresponding population one year in the future. The estimated closest strain to the future was identified by either the best model in the validation period (mutational load and LBI) or the best model in the test period
(HI antigenic novelty and mutational load).

In [ ]:
validation_tip_attributes_path = snakemake.input.validation_tip_attributes
validation_forecasts_path = snakemake.input.validation_forecasts_path

test_tip_attributes_path = snakemake.input.test_tip_attributes
test_forecasts_path = snakemake.input.test_forecasts_path

vaccines_json_path = snakemake.input.vaccines_json_path

output_figure = snakemake.output.figure

In [ ]:
"""
validation_tip_attributes_path = "../results/builds/natural/natural_sample_1_with_90_vpm_sliding/tip_attributes_with_weighted_distances.tsv"
validation_forecasts_path = "../results/builds/natural/natural_sample_1_with_90_vpm_sliding/forecasts.tsv"

test_tip_attributes_path = "../results/builds/natural/natural_sample_1_with_90_vpm_sliding_test_tree/tip_attributes_with_weighted_distances.tsv"
test_forecasts_path = "../results/builds/natural/natural_sample_1_with_90_vpm_sliding_test_tree/forecasts.tsv"

vaccines_json_path = "../config/vaccines_h3n2.json"

output_figure = "../manuscript/figures/vaccine-comparison.pdf"
"""

In [ ]:
import json
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import numpy as np
import pandas as pd
from pandas.plotting import register_matplotlib_converters
from scipy.stats import pearsonr, spearmanr, probplot
import seaborn as sns
import statsmodels.api as sm

%matplotlib inline

In [ ]:
register_matplotlib_converters()

In [ ]:
first_validation_timepoint = "2003-10-01"

min_clade_frequency = 0.15
precision = 4
pseudofrequency = 0.001
number_of_bootstrap_samples = 200

In [ ]:
sns.set_style("ticks")

In [ ]:
# Display figures at a reasonable default size.
mpl.rcParams['figure.figsize'] = (6, 4)

# Disable top and right spines.
mpl.rcParams['axes.spines.top'] = False
mpl.rcParams['axes.spines.right'] = False
    
# Display and save figures at higher resolution for presentations and manuscripts.
mpl.rcParams['savefig.dpi'] = 200
mpl.rcParams['figure.dpi'] = 200

# Display text at sizes large enough for presentations and manuscripts.
mpl.rcParams['font.weight'] = "normal"
mpl.rcParams['axes.labelweight'] = "normal"
mpl.rcParams['font.size'] = 18
mpl.rcParams['axes.labelsize'] = 14
mpl.rcParams['legend.fontsize'] = 12
mpl.rcParams['xtick.labelsize'] = 14
mpl.rcParams['ytick.labelsize'] = 14

mpl.rc('text', usetex=False)

## Load data

In [ ]:
validation_forecasts = pd.read_csv(
    validation_forecasts_path,
    sep="\t",
    parse_dates=["timepoint", "future_timepoint"]
)

In [ ]:
validation_forecasts = validation_forecasts.query("timepoint >= '%s'" % first_validation_timepoint).dropna().copy()

In [ ]:
test_forecasts = pd.read_csv(
    test_forecasts_path,
    sep="\t",
    parse_dates=["timepoint", "future_timepoint"]
).dropna()

In [ ]:
test_forecasts.head()

In [ ]:
validation_forecasts.tail()

In [ ]:
forecasts = pd.concat([validation_forecasts, test_forecasts])

In [ ]:
forecasts.head()

In [ ]:
forecasts.shape

In [ ]:
validation_forecasts.shape

In [ ]:
test_forecasts.shape

In [ ]:
validation_forecasts.shape[0] + test_forecasts.shape[0]

In [ ]:
forecasts.dropna().shape

## Estimated and observed closest strains per timepoint

In [ ]:
sorted_df = forecasts.sort_values(
    ["timepoint"]
).copy()

In [ ]:
sorted_df.head()

In [ ]:
validation_tips_with_sequence = pd.read_csv(
    validation_tip_attributes_path,
    sep="\t",
    parse_dates=["timepoint"],
    usecols=["strain", "timepoint", "frequency", "aa_sequence"]
)

In [ ]:
test_tips_with_sequence = pd.read_csv(
    test_tip_attributes_path,
    sep="\t",
    parse_dates=["timepoint"],
    usecols=["strain", "timepoint", "frequency", "aa_sequence"]
)

In [ ]:
validation_tips_with_sequence.shape

In [ ]:
validation_tips_with_sequence = validation_tips_with_sequence[
    validation_tips_with_sequence["timepoint"].isin(validation_forecasts["timepoint"])
].copy()

In [ ]:
validation_tips_with_sequence.shape

In [ ]:
distinct_validation_tips_with_sequence = validation_tips_with_sequence.groupby(["timepoint", "aa_sequence"]).first().reset_index()

In [ ]:
distinct_test_tips_with_sequence = test_tips_with_sequence.groupby(["timepoint", "aa_sequence"]).first().reset_index()

In [ ]:
distinct_validation_tips_with_sequence.head()

In [ ]:
distinct_validation_tips_with_sequence.tail()

In [ ]:
distinct_test_tips_with_sequence.head()

In [ ]:
distinct_test_tips_with_sequence.tail()

In [ ]:
tips_with_sequence = pd.concat([validation_tips_with_sequence, test_tips_with_sequence])

In [ ]:
distinct_tips_with_sequence = pd.concat([distinct_validation_tips_with_sequence, distinct_test_tips_with_sequence])

In [ ]:
sorted_df = sorted_df.merge(
    distinct_tips_with_sequence,
    on=["timepoint", "strain", "frequency"]
)

In [ ]:
sorted_df.head()

In [ ]:
sorted_df.tail()

In [ ]:
sorted_df["timepoint_rank"] = sorted_df.groupby("timepoint")["weighted_distance_to_future"].rank(pct=True)
sorted_df["timepoint_estimated_rank"] = sorted_df.groupby("timepoint")["y"].rank(pct=True)

In [ ]:
spearmanr(
    sorted_df["timepoint_rank"],
    sorted_df["timepoint_estimated_rank"]
)

In [ ]:
observed_closest_strains = sorted_df.sort_values(
    ["timepoint", "weighted_distance_to_future"],
    ascending=True
).groupby("timepoint").first().reset_index().loc[:, ["timepoint", "strain", "weighted_distance_to_future", "aa_sequence"]]

In [ ]:
estimated_closest_strains = sorted_df.sort_values(
    ["timepoint", "y"],
    ascending=True
).groupby("timepoint").first().reset_index().loc[:, ["timepoint", "strain", "weighted_distance_to_future", "y", "timepoint_rank", "aa_sequence"]]

In [ ]:
estimated_closest_strains.head()

In [ ]:
closest_strains = observed_closest_strains.merge(
    estimated_closest_strains,
    on=["timepoint"],
    suffixes=["_observed", "_estimated"]
)

In [ ]:
closest_strains.head()

In [ ]:
with open(vaccines_json_path, "r") as fh:
    vaccines_json = json.load(fh)

In [ ]:
vaccine_df = pd.DataFrame([
    {
        "vaccine_strain": vaccine,
        "vaccine_timepoint": vaccine_data["vaccine"]["timepoint"],
        "aa_sequence": vaccine_data["aa_sequence"]
    }
    for vaccine, vaccine_data in vaccines_json["nodes"].items()
])
vaccine_df["vaccine_timepoint"] = pd.to_datetime(vaccine_df["vaccine_timepoint"])
vaccine_df["future_vaccine_timepoint"] = vaccine_df["vaccine_timepoint"] + pd.DateOffset(months=12)

In [ ]:
vaccine_df

In [ ]:
forecasts.head()

In [ ]:
tips_for_vaccines_df = vaccine_df.merge(
    tips_with_sequence,
    left_on=["future_vaccine_timepoint"],
    right_on=["timepoint"],
    suffixes=["_vaccine", ""]
)

In [ ]:
tips_for_vaccines_df.head()

In [ ]:
def calculate_weighted_distance_between_pairs(row):
    distance = (
        np.frombuffer(row["aa_sequence_vaccine"].encode(), dtype="S1") !=
        np.frombuffer(row["aa_sequence"].encode(), dtype="S1")
    ).sum()
    return row["frequency"] * distance

def calculate_weighted_distance_by_group(group_df):
    return group_df.apply(calculate_weighted_distance_between_pairs, axis=1).sum()

In [ ]:
vaccine_distance_to_future = tips_for_vaccines_df.groupby("vaccine_timepoint").apply(
    calculate_weighted_distance_by_group
).reset_index(name="vaccine_distance_to_future")

In [ ]:
vaccine_forecasts = vaccine_df.merge(
    vaccine_distance_to_future,
    on="vaccine_timepoint"
)

In [ ]:
vaccine_forecasts

In [ ]:
closest_strains_with_vaccine_timepoints = closest_strains[closest_strains["timepoint"].isin(vaccine_forecasts["vaccine_timepoint"])]

In [ ]:
vaccine_forecasts = vaccine_forecasts[vaccine_forecasts["vaccine_timepoint"].isin(closest_strains_with_vaccine_timepoints["timepoint"])].copy()

In [ ]:
closest_strains_with_vaccine_timepoints.shape

In [ ]:
vaccine_forecasts.shape

In [ ]:
for index, record in vaccine_forecasts.iterrows():
    print(record["vaccine_strain"])

In [ ]:
max_analysis_distance_to_future = closest_strains_with_vaccine_timepoints.loc[
    :, ["weighted_distance_to_future_observed", "weighted_distance_to_future_estimated"]
].max().max()

max_vaccine_distance_to_future = vaccine_forecasts["vaccine_distance_to_future"].max()
max_distance_to_future = max(max_analysis_distance_to_future, max_vaccine_distance_to_future)

In [ ]:
distance_ticks = np.arange(int(np.ceil(max_distance_to_future)) + 2, step=2)

In [ ]:
distance_ticks

In [ ]:
vaccine_forecasts["vaccine_distance_to_future"].max()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 4))
strain_alpha=0.5
ax.plot(
    closest_strains_with_vaccine_timepoints["timepoint"],
    closest_strains_with_vaccine_timepoints["weighted_distance_to_future_observed"],
    "o-", label="Observed best", alpha=strain_alpha
)
ax.plot(
    closest_strains_with_vaccine_timepoints["timepoint"],
    closest_strains_with_vaccine_timepoints["weighted_distance_to_future_estimated"],
    "o-", label="Estimated best", alpha=strain_alpha
)
ax.plot(
    vaccine_forecasts["vaccine_timepoint"],
    vaccine_forecasts["vaccine_distance_to_future"],
    "o-", label="Vaccine strain", alpha=strain_alpha
)

for index, record in vaccine_forecasts.iterrows():
    ax.text(
        record["vaccine_timepoint"],
        record["vaccine_distance_to_future"],
        record["vaccine_strain"],
        fontsize=8
    )

ax.axvline(
    validation_forecasts["timepoint"].max(),
    zorder=-10,
    alpha=0.2,
    color="#000000",
    linestyle="--",
    label="Last validation timepoint"
)

ax.legend(
    loc=(0.01, 0.7),
    frameon=False
)

ax.set_yticks(distance_ticks)

ax.set_xlabel("Date")
ax.set_ylabel("Weighted distance\nto the future (AAs)")
ax.set_ylim(bottom=0)

plt.tight_layout()
plt.savefig(output_figure, bbox_inches="tight")

In [ ]:
vaccine_forecasts

In [ ]:
merged_model_and_vaccine_df = closest_strains_with_vaccine_timepoints.merge(
    vaccine_forecasts,
    left_on=["timepoint"],
    right_on=["vaccine_timepoint"]
)

In [ ]:
merged_model_and_vaccine_df["vaccine_distance_to_future"] - merged_model_and_vaccine_df["weighted_distance_to_future_estimated"]